#### This assignment may be worked individually or in pairs. Enter your name/s here:
    

In [578]:
# Manojwal Oddiraju; mo24776

# Assignment 2: Decision Trees

In this assignment we'll implement the Decision Tree algorithm to classify patients as either having or not having diabetic retinopathy. For this task we'll be using the Diabetic Retinopathy data set, which contains features from the Messidor image set to predict whether an image contains signs of diabetic retinopathy or not. This dataset has `1150` instances and `20` attributes (some categorical, some continuous). You can find additional details about the dataset [here](http://archive.ics.uci.edu/ml/datasets/Diabetic+Retinopathy+Debrecen+Data+Set).

Attribute Information:

    0) The binary result of quality assessment. 0=bad quality 1=sufficient quality.

    1) The binary result of pre-screening, where 1 indicates severe retinal abnormality and 0 its lack. 

    2-7) The results of MA detection. Each feature value stand for the number of MAs found at the confidence levels alpha = 0.5, . . . , 1, respectively. 

    8-15) Contains the same information as 2-7, but for exudates. However, as exudates are represented by a set of points rather than the number of pixels constructing the lesions, these features are normalized by dividing the number of lesions with the diameter of the region of interest (ROI) to compensate for different image sizes. 

    16) The euclidean distance between the center of the macula and the center of the optic disc. This feature is also normalized with the diameter of the ROI.

    17) The diameter of the optic disc. 

    18) Result of the AM/FM-based (amplitude-modulation frequency-modulation) imaging. 0=normal and 1=abnormal.

    19) Class label. 1 = contains signs of Diabetic Retinopathy, 0 = no signs of Diabetic Retinopathy.

#### Implementation: 
The function prototypes are given to you, please don't change these. You can add additional helper functions if needed. 

*Suggestion:* The dataset is substantially big, for the purpose of easy debugging, work with a subset of the data and test your decision tree implementation on that.

#### Notes:
Parts of this assignment will be **autograded** so a couple of caveats :-
- Entropy is calculated using log with base 2, `math.log2(x)`.
- For continuous features ensure that the threshold value lies exactly between 2 values. For example, if for feature 2 the best split occurs between 10 and 15 then the threshold value will be set as 12.5. For binary features [0/1] the threshold value will be 0.5.
- All values < `thresh_val` go to the left child and all values >= `thresh_val` go to the right child.

In [579]:
# Standard Headers
# You are welcome to add additional headers if you wish
# EXCEPT for scikit-learn... You may NOT use scikit-learn for this assignment!
import pandas as pd
from math import log2
import time

In [580]:
class TreeNode:
    is_leaf = True          # boolean variable to check if the node is a leaf
    feature_idx = None      # index that identifies the feature
    thresh_val = None       # threshold value that splits the node
    prediction = None       # prediction class (only valid for leaf nodes)
    left_child = None       # left TreeNode (all values < thresh_val)
    right_child = None      # right TreeNode (all values >= thresh_val)
    
    def printTree(self, level=0):    # for debugging purposes
        if self.is_leaf:
            print ('-'*level + 'Leaf Node:      predicts ' + str(self.prediction))
        else:
            print ('-'*level + 'Internal Node:  splits on feature ' 
                   + str(self.feature_idx) + ' with threshold ' + str(self.thresh_val))
            self.left_child.printTree(level+1)
            self.right_child.printTree(level+1)

Q1. Implement the function `make_prediction` that takes the decision tree root and a data point instance and returns the prediction label.

In [581]:
def make_prediction(tree_root, data_point):
    # If the node is a leaf, return the prediction at this node
    if tree_root.is_leaf:
        # print(f"Leaf node reached: prediction = {tree_root.prediction}")
        return tree_root.prediction 
     
    # Print the feature index, threshold, and feature value to debug
    feature_value = data_point[tree_root.feature_idx] 
    # print(f"At node: feature {tree_root.feature_idx}, threshold = {tree_root.thresh_val}, feature value = {feature_value}")
    
    # Check the direction of the split (left or right)
    if feature_value < tree_root.thresh_val: 
        # print(f"Going left: feature value ({feature_value}) < threshold ({tree_root.thresh_val})")
        if tree_root.left_child is not None:
            return make_prediction(tree_root.left_child, data_point) 
        else:
            return tree_root.prediction  # Return the prediction at the current node if no left child
    
    else:
         # print(f"Going right: feature value ({feature_value}) >= threshold ({tree_root.thresh_val})")
        if tree_root.right_child is not None: 
            return make_prediction(tree_root.right_child, data_point) 
        else:
            return tree_root.prediction  # Return the prediction at the current node if no right child


Q2. Implement the function `split_dataset` given an input data set, a `feature_idx` and the `threshold` for the feature. `left_split` will have all values < `threshold` and `right_split` will have all values >= `threshold`.

In [582]:
def split_dataset(data, feature_idx, threshold):
    left_split = data[data.iloc[:, feature_idx] < threshold]
    right_split = data[data.iloc[:, feature_idx] >= threshold]
    return (left_split, right_split)

Q3. Implement the function `calc_entropy` to return the entropy of the input dataset.

In [583]:
def calc_entropy(data):
    label_index = data.shape[1] - 1 
    labels = data.iloc[:, label_index]
    # print("Len data is ", len(data))
    if len(data) == 0:
        return 0
    positive_log = (labels[labels == 1].count() / len(data))
    if positive_log == 0 or positive_log == 1:
        return 0 
    entropy = -(positive_log * log2(positive_log) + (1 - positive_log) * log2(1 - positive_log))
    return entropy


Q4. Implement the function `calc_best_threshold` which returns the best information gain and the corresponding threshold value for one feature at `feature_idx`. If there is a tie between threshold values, return the lowest threshold value.

In [584]:
def calc_best_threshold(data, feature_idx):

    feature_column = data.iloc[:, feature_idx]
    sorted_features = feature_column.sort_values() 
     
    best_info_gain = 0.0
    best_thresh = None
    og_entropy = calc_entropy(data)

    # Iterate through sorted feature values
    for i in range(1, len(sorted_features)): 
        threshold = (sorted_features.iloc[i - 1] + sorted_features.iloc[i]) / 2
        left_split = data[data.iloc[:, feature_idx] < threshold]
        right_split = data[data.iloc[:, feature_idx] >= threshold]
        left_entropy = calc_entropy(left_split)
        right_entropy = calc_entropy(right_split)

        # Calculate final entropy and information gain
        final_entropy = ((len(left_split) / len(data)) * left_entropy) + ((len(right_split) / len(data)) * right_entropy)
        info_gain = og_entropy - final_entropy

        if info_gain > best_info_gain:
            best_info_gain = info_gain
            best_thresh = threshold

    return (best_info_gain, best_thresh)


Q5. Implement the function `identify_best_split` which returns the best feature to split on for an input dataset, and also returns the corresponding threshold value. If there is a tie between features, choose the one with the lowest `feature_idx`.

In [585]:
def identify_best_split(data):
    if len(data) < 2: 
        return (None, None) 
    best_feature = None
    best_thresh = None 
    best_info_gain = 0.0
         
    for feature_idx in range(data.shape[1] - 1): 
        info_gain, thresh = calc_best_threshold(data, feature_idx)
         
        if info_gain > best_info_gain:  
            best_info_gain = info_gain
            best_feature = feature_idx  
            best_thresh = thresh 
    return (best_feature, best_thresh)

Q6. Implement the function `create_leaf_node` which returns a `TreeNode` with `is_leaf=True` and `prediction` set to whichever classification occurs most in the dataset at this node. If there is a tie, choose classification label 1 (has disease). 

In [586]:
def create_leaf_node(data):     
    # Get the labels (assuming the last column is the label)
    labels = data.iloc[:, -1]
    
    # Count the occurrences of each label
    feature_counts = labels.value_counts()

    # Find the most common label (ties are automatically handled by pandas value_counts)
    most_common_label = feature_counts.idxmax()

    # Create a leaf node and set the prediction
    node = TreeNode()
    node.is_leaf = True  # Ensure is_leaf is set to True for leaf nodes
    node.prediction = most_common_label
    return node


Q7. Implement the `create_decision_tree` function. `max_levels` denotes the maximum height of the tree. For example, if `max_levels = 1` then the decision tree will only contain the leaf node at the root. 

[Hint: this is where the recursion happens.]

In [587]:
def create_decision_tree(data, max_levels):
    # If max levels is 0 or there is no data, return a leaf node
    if max_levels == 0 or len(data) == 0:
        return create_leaf_node(data)

    # Check if all labels are the same (pure node)
    labels = data.iloc[:, -1]
    if len(set(labels)) == 1:
        return create_leaf_node(data)

    # Identify the best feature and threshold for splitting
    best_feature, best_thresh = identify_best_split(data)

    # If no valid split found, create a leaf node
    if best_feature is None or best_thresh is None:
        return create_leaf_node(data)

    # Split the data based on the best feature and threshold
    left_data = data[data.iloc[:, best_feature] < best_thresh]
    right_data = data[data.iloc[:, best_feature] >= best_thresh]

    # If one of the branches is empty, return a leaf node
    if left_data.empty or right_data.empty:
        return create_leaf_node(data)

    # Create the current node and set the splitting feature and threshold
    current_node = TreeNode()
    current_node.is_leaf = False  # Explicitly setting is_leaf for internal nodes
    current_node.feature_idx = best_feature
    current_node.thresh_val = best_thresh

    # Recursively create left and right children
    current_node.left_child = create_decision_tree(left_data, max_levels - 1) 
    current_node.right_child = create_decision_tree(right_data, max_levels - 1)

    return current_node


Q8. Given a decision tree and a test set, the function `calc_accuracy` returns the accuracy of the classifier on a test set. Return the accuracy as a decimal (i.e. return 0.987 and *not* 98.7).

In [588]:
def calc_accuracy(tree_root, test_data):
    correct_predictions = 0 
    total_predictions = len(test_data)

    for index, instance in test_data.iterrows():
        # print('inside calc_accuracy')
        # print('printing it here in calc_accuracy' , tree_root)
        # print('the type of instance is: ', type(instance))
        predicted_label = make_prediction(tree_root, instance)
        # print("predicted label:" , predicted_label)
        true_label = instance.iloc[-1]
        if true_label == predicted_label:
            correct_predictions += 1 

    return correct_predictions / total_predictions 

Q9. Now measure the accuracy of using a decision tree on this data with a 5-fold cross validation. 
Set the `max_levels` parameter to 10 and return the average accuracy from a 5-fold-CV (as a decimal).

This must run in under 10 minutes, otherwise points will be deducted. 

In [ ]:
def run_CV(filename):
    start = time.time()
    d = pd.read_csv(filename, header = None)
    accuracy = 0
    fold_size = len(d) // 5
    accuracies = []

    for i in range(5):
        test_set = d[i * fold_size: (i + 1) * fold_size]
        train_set = pd.concat([d.iloc[:i * fold_size], d.iloc[(i + 1) * fold_size:]]) 
        tree = create_decision_tree(train_set, max_levels=10)
        fold_accuracy = calc_accuracy(tree, test_set)
        print("fold accuracy", fold_accuracy)
        accuracies.append(fold_accuracy)

    accuracy = sum(accuracies) / len(accuracies)
    #print runtime
    end = time.time()
    total_time = end - start
    min = int(total_time // 60)
    sec = int(total_time % 60)
    print(f"Time taken: {min}:{sec:02d}")

    return accuracy


accuracy = run_CV("messidor_features.txt")
print('Accuracy:', accuracy*100)